IF NO STARTUP SCRIPT WAS USED, run in terminal:

<code>bash text2video_startup.txt</code>

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

from stable_diffusion_videos import StableDiffusionWalkPipeline
import torch
import mediapy as media

import os
from dotenv import load_dotenv
from pyuploadcare import Uploadcare, File
import psycopg2

load_dotenv()

UC_public_key = os.getenv('UPLOADCARE_PUBLIC_KEY')
UC_secret_key = os.getenv('UPLOADCARE_PRIVATE_KEY')

def get_db_connection():
    conn = psycopg2.connect(
        dbname=os.getenv('DB_DBNAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT'),
    )
    return conn

In [ ]:
model_id = "nitrosocke/elden-ring-diffusion" 
# ["stabilityai/stable-diffusion-2-1", 
# "stabilityai/stable-diffusion-2-base", 
# "runwayml/stable-diffusion-v1-5", 
# "SG161222/Realistic_Vision_V1.3", 
# "prompthero/openjourney", 
# "jzli/DreamShaper-3.32", 
# "nitrosocke/redshift-diffusion",
# "nitrosocke/elden-ring-diffusion", 
# "nitrosocke/Future-Diffusion", 
# "nitrosocke/mo-di-diffusion" ]


pipeline = StableDiffusionWalkPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
).to("cuda")

# Disable Safety Filters Treu/False
disable_safety_filter = False 

if disable_safety_filter == True:
  def dummy(images, **kwargs):
      return images, False
  pipeline.safety_checker = dummy


In [ ]:
prompts=['An intricate masterpiece painting by Monet showing a landscape of rolling hills with little villages in between, sunset, professional majestic oil painting by Ed Blinkey, Atey Ghailan, Studio Ghibli, by Jeremy Mann, Greg Manchess, Antonio Moro, trending on ArtStation, trending on CGSociety, Intricate, High Detail, Sharp focus, dramatic, photorealistic painting art by midjourney and greg rutkowski', 
        'An intricate masterpiece painting by Cezanne showing an urban landscape of skyscrapers and highways, sunset, professional majestic oil painting by Ed Blinkey, Atey Ghailan, Studio Ghibli, by Jeremy Mann, Greg Manchess, Antonio Moro, trending on ArtStation, trending on CGSociety, Intricate, High Detail, Sharp focus, dramatic, photorealistic painting art by midjourney and greg rutkowski'],
seeds=[10, 11],
num_interpolation_steps=5,
height=512,  # use multiples of 64 if > 512. Multiples of 8 if < 512.
width=1024,   # use multiples of 64 if > 512. Multiples of 8 if < 512.
output_dir='outputs',        # Where images/videos will be saved
name='test3',        # Subdirectory of output_dir where images/videos will be saved
guidance_scale=9,         # Higher adheres to prompt more, lower lets model take the wheel
num_inference_steps=50,     # Number of diffusion steps per image generated. 50 is good default
upsample = True

In [ ]:
!python run gen_video.py

In [ ]:
video_path = pipeline.walk(prompts, seeds, num_interpolation_steps, height, width, output_dir, name, guidance_scale, num_inference_steps, upsample)

uploadcare = Uploadcare(UC_public_key, UC_secret_key)
uploadcareFile = uploadcare.upload(video_path)
print(uploadcareFile)


with open(video_path, 'rb') as file_object:
     uc_file: File = uploadcare.upload(file_object, store=True)
     print(uc_file.info)
     print(uc_file.original_file_url)

# ### Database reading and Writing
# conn = get_db_connection()
# cur = conn.cursor()


# ##################This is not yet working########## Table not yet created, variables not yet defined

# # Insert data into the table
# cur.execute('INSERT INTO ai_headlines_table (headline, paragraph, image_url)'
#                'VALUES (%s, %s, %s)',
#                (textAnswer[0],
#                textAnswer[1],
#                imgLink)
#                )

# conn.commit()
# cur.close()
# conn.close()

# print("Generation finished, data inserted to database")

video = media.read_video(video_path)
media.show_video(video, fps=6) 